**SOR Method**

Downloading matrix

In [1]:
import requests

matrix_id = "HB/bcsstk16"  # choose one of the IDs above
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = matrix_id.split("/")[-1] + ".mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status()

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk16.mat


Loading Matrix

In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
data = loadmat("bcsstk16.mat")

# Access the matrix stored in the 'Problem' struct
A = data['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_sparse = csr_matrix(A)

# Load RHS vector if available
if 'b' in data['Problem'][0, 0].dtype.names:
    b = data['Problem'][0, 0]['b'].flatten()
else:
    b = np.random.rand(A_sparse.shape[0])  # Generate dummy RHS

# Inspect matrix
print("Shape:", A_sparse.shape)
print("Non-zeros:", A_sparse.nnz)

Shape: (4884, 4884)
Non-zeros: 290378


9. SOR Method
  
    Matrix Requirement: Square, Diagonally dominant or SPD

In [3]:
import numpy as np

def sor(A, b, omega=1.0, max_iter=1000, tol=1e-10):
    """
    Successive Over-Relaxation (SOR) method.
    A : sparse or dense matrix
    b : RHS vector
    omega : relaxation parameter (0 < omega < 2)
    """
    # Convert sparse to dense for simplicity
    A = np.array(A.todense(), dtype=float)
    n = A.shape[0]
    x = np.zeros(n)  # initial guess

    for it in range(max_iter):
        x_new = np.copy(x)

        for i in range(n):
            sum1 = np.dot(A[i, :i], x_new[:i])   # already updated values
            sum2 = np.dot(A[i, i+1:], x[i+1:])   # old values
            x_new[i] = (1 - omega) * x[i] + (omega / A[i, i]) * (b[i] - sum1 - sum2)

        # Convergence check
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            return x_new, it+1, np.linalg.norm(b - A @ x_new)

        x = x_new

    return x, max_iter, np.linalg.norm(b - A @ x)

# Example usage
b = np.random.rand(A_sparse.shape[0])  # RHS vector
x, iterations, residual = sor(A_sparse, b, omega=1.25)  # omega > 1 accelerates convergence

print("SOR solution (first 5 entries):", x[:5])
print("Iterations:", iterations)
print("Final residual norm:", residual)

SOR solution (first 5 entries): [ 1.95056226e-08 -8.07916508e-09  6.81771963e-01  2.12412003e-08
 -6.47950086e-09]
Iterations: 340
Final residual norm: 0.17783112579322116
